In [7]:
import os
import fileinput
from schnetpack.datasets import QM9
import numpy as np
import pandas as pd
from xlwt import Workbook


#Load qm9data
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

#Input range of QM9 to be labelled
print('how many QM9 molecules do you want to label?')
#number_of_molecules = input()
#number_of_molecules = int(number_of_molecules)

how many QM9 molecules do you want to label?


In [8]:
def get_coordinates(props):
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    return x, y, z

def write_coordinates_xyz(name_xyz,props):    
    x = get_coordinates(props)[0]
    y = get_coordinates(props)[1]
    z = get_coordinates(props)[2] 
    file_xyz = open(name_xyz, mode='w',encoding='utf-8')
    
    number_of_atoms = len(z)
    number_of_atoms = int(number_of_atoms)
    file_xyz.write(str(len(z))+'\n')    
    file_xyz.write('title'+'\n')
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            string = ('H' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 6:
            string = ('C' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 8:
            string = ('O' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 7:
            string = ('N' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 9:
            string = ('F' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
    file_xyz.close()

def store_positions(name_mol,element,idx):
    countmollines = 0
    array_of_element = []
    for line in fileinput.FileInput(name_mol,inplace=0):
        countmollines = countmollines + 1
        if element in line:
            array_of_element.append([])
            array_of_element[len(array_of_element)-1] = int(countmollines-4)
    return array_of_element

def connection_matrix(name_mol,array_of_element,number_atoms):
    count=0
    connected_positions = []
    array = []
    for j in range(len(array_of_element)):
        connected_positions.append([])
        array.append([])
        array[j] = str(array_of_element[j])
    for line in fileinput.FileInput(name_mol,inplace=0):
        count = count + 1
        if count >= number_atoms + 5:
            line = line.replace(line,line[:7])
            for j in range(len(array_of_element)):
                if ' ' + str(array_of_element[j]) + ' ' in line and 'RAD' not in line:
                    connected_positions[j].append([])
                    if str(line[1]+line[2]+line[3]) != ' '+array[j]+' ':
                        connected = str(line[1:3])
                        connected = int(connected)
                        connected_positions[j][len(connected_positions[j])-1]=connected
                    else:
                        connected = str(line[4]+line[5]+line[6])
                        connected = int(connected)
                        connected_positions[j][len(connected_positions[j])-1]=connected
#    print(connected_positions)
    return connected_positions

def neighboring_connections(name_mol,carbon_position,number_atoms):
    carbon_neighbor_connections = []
    countmollines = 0
    for line in fileinput.FileInput(name_mol,inplace=0):
        countmollines = countmollines + 1
        if countmollines >= number_atoms + 5:
            line = line.replace(line,line[:7])
            if ' ' + str(carbon_position) + ' ' in line and 'RAD' not in line:
                carbon_neighbor_connections.append([])
                if str(line[1]+line[2]+line[3]) != ' ' + str(carbon_position) + ' ':
                    connected = str(line[1]+line[2]+line[3])
                    connected = int(connected)
                    carbon_neighbor_connections[len(carbon_neighbor_connections)-1]=connected
                else:
                    connected = str(line[4]+line[5]+line[6])
                    connected = int(connected)
                    carbon_neighbor_connections[len(carbon_neighbor_connections)-1]=connected
    return carbon_neighbor_connections
                    

In [10]:
wb1 = Workbook()
sheet1 = wb1.add_sheet('Sheet1')
row=0

# Store an array of all oxygen positions in each molecules
for idx in range(3000):
    at, props = qm9data.get_properties(idx)
    
    print(idx)
    name_xyz = '../data/label/3000/' + str(idx) + '.xyz'
    name_mol = '../data/label/3000/' + str(idx) + '.mol'
    
    #write coordinates xyz file
    write_coordinates_xyz(name_xyz,props)
    
    
    #convert xyz file to mol file using obabel
    os.system('obabel ' + name_xyz + ' -O ' + name_mol)
    

    element = ' O  '
    array_of_element = store_positions(name_mol,element,idx)
    print(array_of_element)
    
    x = get_coordinates(props)[0]
    number_atoms = len(x)
    
    # construct the matrix of connection
    if 8 in props['_atomic_numbers']:
        positions = array_of_element
        connected_positions = connection_matrix(name_mol,positions,number_atoms)
        print(connected_positions)
        
        ##Initialize important data holders 
        countC = np.zeros((len(connected_positions)))
        countH = np.zeros((len(connected_positions)))
        group = np.zeros((len(connected_positions)))
        label = np.zeros((len(connected_positions)))
        
        for j in range(len(connected_positions)):
            for i in range(len(connected_positions[j])):
                countxyzlines=0
                for line2 in fileinput.FileInput(name_xyz,inplace=0):
                    countxyzlines = countxyzlines+1
                    if countxyzlines == connected_positions[j][i]+2:
                        if 'C ' in line2:
                            countC[j] = countC[j]+1
                        if 'H ' in line2:
                            countH[j] = countH[j]+1
                            
        #label between alcohol/carbonyl/ether
        
            if countH[j] > 0:
                group[j] = 1.0
            elif countC[j] > 0 and countH[j] == 0:
                if countC[j] == 1:
                    group[j] = 2.0
                if countC[j] == 2:
                    label[j] = 1
                    

            if countH[j] > 0:
                if connected_positions[j][0] < connected_positions[j][1]:
                    connected_positions[j][0] = connected_positions[j][0]
                else:
                    connected_positions[j][0] = connected_positions[j][1]
           
            carbon_position = connected_positions[j][0]        

            
            if group[j] == 2.0:
                countC1 = np.zeros((len(connected_positions)))
                countO1 = np.zeros((len(connected_positions)))
                countH1 = np.zeros((len(connected_positions)))
                countN1 = np.zeros((len(connected_positions)))
                carbon_neighbor_connections = neighboring_connections(name_mol,carbon_position,number_atoms)
                for i in range(len(carbon_neighbor_connections)):
                    countxyzlines = 0
                    for line2 in fileinput.FileInput(name_xyz,inplace=0):
                        countxyzlines = countxyzlines + 1
                        if countxyzlines == carbon_neighbor_connections[i] + 2:
                            if 'C ' in line2:
                                countC1[j] = countC1[j] + 1
                            if 'H ' in line2:
                                countH1[j] = countH1[j] + 1
                            if 'N ' in line2:
                                countN1[j] = countN1[j] + 1
                            if 'O ' in line2:
                                countO1[j] = countO1[j] +1
                
                if countH1[j] == 2 and countC1[j] == 0:
                    label[j] = 2
                elif countC1[j] == 1 and countO1[j] ==1 and countH1[j] > 0 and countN1[j] ==0:
                    label[j] = 2
                elif countC1[j] == 2:
                    label[j] = 3
                elif countO1[j] > 1:
                    label[j] = 4
                elif countN1[j] > 0:
                    label[j] = 5
                else:
                    label[j] = 6
                
            
            
        
            if group[j] == 1.0:
                countC2 = np.zeros((len(connected_positions)))
                countH2 = np.zeros((len(connected_positions)))
                countO2 = np.zeros((len(connected_positions)))
                countN2 = np.zeros((len(connected_positions)))
                carbon_neighbor_connections = neighboring_connections(name_mol,carbon_position,number_atoms)
                for i in range(len(carbon_neighbor_connections)):
                    countxyzlines = 0
                    for line2 in fileinput.FileInput(name_xyz,inplace=0):
                        countxyzlines = countxyzlines + 1
                        if countxyzlines == carbon_neighbor_connections[i] + 2:
                            if 'C ' in line2:
                                countC2[j] = countC2[j] + 1
                            if 'H ' in line2:
                                countH2[j] = countH2[j] + 1
                            if 'N ' in line2:
                                countN2[j] = countN2[j] + 1
                            if 'O ' in line2:
                                countO2[j] = countO2[j] + 1
                if countC[j] == 0:
                    label[j] = 7
                elif countC2[j] + countN2[j] + countO2[j] == 2:
                    label[j] = 7
                elif countH2[j] == 2 or countH2[j] == 3:  
                    label[j]=7
                elif countC2[j] + countN2[j] + countO2[j] == 3:
                    label[j] = 8
                elif countC2[j] + countN2[j] + countO2[j] == 4:
                    label[j] = 9
                else:
                    label[j] = 6
                print(countC2[j])
                print(countH2[j])
                print(countN2[j])
                print(countO2[j])
                print(label[j])
            
            sheet1.write(row,0,label[j])
            sheet1.write(row,1,idx)
            row=row+1

wb1.save('../data/label/3000/label-o.xls')
read_file = pd.read_excel(r'../data/label/3000/label-o.xls')
read_file.to_csv(r'../data/label/3000/label-o.csv',index=None, header=True)
os.system('rm ../data/label/3000/*mol')
os.system('rm ../data/label/3000/*xyz')
            
    
            
                               
                
                
    
    
    
    
#    element = ' H  '
#    store_positions(name_mol,element,idx)
    

0
[]
1
[]
2
[1]
[[2, 3]]
0.0
0.0
0.0
1.0
7.0
3
[]
4
[]
5
[2]
[[1]]
6
[]
7
[2]
[[1, 6]]
0.0
3.0
0.0
1.0
7.0
8
[]
9
[]
10
[3]
[[2]]
11
[3]
[[2]]
12
[]
13
[3]
[[2, 9]]
1.0
2.0
0.0
1.0
7.0
14
[2]
[[1, 3]]
15
[]
16
[3]
[[2, 1]]
17
[4]
[[2]]
18
[4]
[[2]]
19
[4]
[[2]]
20
[]
21
[4]
[[2, 12]]
2.0
1.0
0.0
1.0
8.0
22
[]
23
[]
24
[]
25
[1]
[[2]]
26
[1]
[[2]]
27
[1, 4]
[[2], [3]]
28
[]
29
[]
30
[]
31
[]
32
[1]
[[2, 5]]
1.0
2.0
0.0
1.0
7.0
33
[1]
[[2, 5]]
1.0
2.0
0.0
1.0
7.0
34
[4]
[[3]]
35
[4]
[[3]]
36
[2, 4]
[[1, 3], [3]]
37
[1, 4]
[[2, 5], [3]]
1.0
2.0
0.0
1.0
7.0
38
[]
39
[4]
[[12, 3]]
1.0
2.0
0.0
1.0
7.0
40
[3]
[[2, 4]]
41
[1, 4]
[[2, 5], [10, 3]]
1.0
2.0
0.0
1.0
7.0
1.0
2.0
0.0
1.0
7.0
42
[]
43
[4]
[[2, 3]]
44
[]
45
[1]
[[2, 5]]
2.0
1.0
0.0
1.0
8.0
46
[]
47
[3]
[[2, 4]]
48
[5]
[[4, 12]]
1.0
0.0
0.0
1.0
7.0
49
[]
50
[]
51
[1]
[[2, 5]]
52
[1]
[[2, 5]]
53
[]
54
[5]
[[2, 15]]
3.0
0.0
0.0
1.0
9.0
55
[3]
[[2]]
56
[3]
[[2]]
57
[3]
[[2]]
58
[3, 5]
[[2], [4]]
59
[3, 5]
[[2], [4]]
60
[]
61
[]
62
[]
63
[

[5]
[[3, 6]]
384
[4]
[[3, 13]]
2.0
1.0
0.0
1.0
8.0
385
[4, 6]
[[3, 13], [5, 16]]
2.0
1.0
0.0
1.0
8.0
1.0
2.0
0.0
1.0
7.0
386
[2, 6]
[[1, 3], [5, 16]]
1.0
2.0
0.0
1.0
7.0
387
[1, 4, 6]
[[2, 7], [3, 11], [5, 14]]
1.0
2.0
0.0
1.0
7.0
2.0
1.0
0.0
1.0
8.0
1.0
2.0
0.0
1.0
7.0
388
[]
389
[]
390
[]
391
[]
392
[4]
[[2, 3]]
393
[4]
[[2, 3]]
394
[]
395
[1]
[[2, 7]]
3.0
0.0
0.0
1.0
9.0
396
[1]
[[2, 7]]
3.0
0.0
0.0
1.0
9.0
397
[6]
[[5]]
398
[6]
[[5]]
399
[4, 6]
[[2, 3], [5]]
400
[1, 6]
[[2, 7], [5]]
3.0
0.0
0.0
1.0
9.0
401
[4]
[[3, 12]]
1.0
2.0
0.0
1.0
7.0
402
[4]
[[3, 12]]
1.0
2.0
0.0
1.0
7.0
403
[4, 6]
[[3, 12], [2, 5]]
1.0
2.0
0.0
1.0
7.0
404
[]
405
[6]
[[5, 3]]
406
[4]
[[12, 3]]
3.0
0.0
0.0
1.0
9.0
407
[2]
[[1, 3]]
408
[1, 4]
[[7, 2], [3, 10]]
1.0
2.0
0.0
1.0
7.0
3.0
0.0
0.0
1.0
9.0
409
[6]
[[5]]
410
[6]
[[5]]
411
[4]
[[3, 5]]
412
[4, 6]
[[3, 5], [5]]
413
[6]
[[5]]
414
[3, 6]
[[2, 4], [5]]
415
[6]
[[5]]
416
[4]
[[5, 3]]
417
[4, 6]
[[5, 3], [5]]
418
[1, 6]
[[2, 7], [5]]
2.0
1.0
0.0
1.0
8.0
419
[

[7]
[[6, 13]]
1.0
0.0
0.0
1.0
7.0
780
[5, 7]
[[4], [6, 14]]
1.0
0.0
0.0
1.0
7.0
781
[5, 7]
[[4, 15], [6, 16]]
1.0
2.0
0.0
1.0
7.0
1.0
0.0
0.0
1.0
7.0
782
[7]
[[6, 18]]
1.0
0.0
0.0
1.0
7.0
783
[2, 7]
[[1, 3], [6, 16]]
1.0
0.0
0.0
1.0
7.0
784
[4, 8]
[[3, 12], [7, 16]]
1.0
0.0
0.0
1.0
7.0
1.0
0.0
0.0
1.0
7.0
785
[4, 7]
[[3, 11], [5]]
1.0
0.0
0.0
1.0
7.0
786
[7]
[[6, 18]]
1.0
0.0
0.0
1.0
7.0
787
[3, 7]
[[2, 12], [6, 16]]
2.0
1.0
0.0
1.0
8.0
1.0
0.0
0.0
1.0
7.0
788
[7]
[[6, 16]]
1.0
0.0
0.0
1.0
7.0
789
[7]
[[6, 15]]
1.0
0.0
0.0
1.0
7.0
790
[1, 6]
[[8, 2], [12, 5]]
1.0
0.0
0.0
1.0
7.0
2.0
1.0
0.0
1.0
8.0
791
[1]
[[2, 8]]
1.0
0.0
0.0
1.0
7.0
792
[5]
[[4, 13]]
1.0
0.0
0.0
1.0
7.0
793
[1, 5]
[[2, 8], [4, 11]]
1.0
2.0
0.0
1.0
7.0
1.0
0.0
0.0
1.0
7.0
794
[7]
[[6, 18]]
1.0
0.0
0.0
1.0
7.0
795
[5, 7]
[[4, 15], [6, 16]]
1.0
2.0
0.0
1.0
7.0
1.0
0.0
0.0
1.0
7.0
796
[7]
[[6, 16]]
1.0
0.0
0.0
1.0
7.0
797
[7]
[[6, 15]]
1.0
0.0
0.0
1.0
7.0
798
[3, 7]
[[2, 4], [6, 14]]
1.0
0.0
0.0
1.0
7.0
799
[1, 7]
[[2, 8

[7]
[[5]]
1093
[7]
[[5]]
1094
[4, 7]
[[5, 2], [5]]
1095
[4, 7]
[[2, 5], [5]]
1096
[3, 7]
[[2, 12], [5]]
2.0
1.0
0.0
1.0
8.0
1097
[3, 7]
[[2, 12], [5]]
2.0
1.0
0.0
1.0
8.0
1098
[]
1099
[7]
[[5, 21]]
2.0
1.0
0.0
1.0
8.0
1100
[4]
[[2, 5]]
1101
[3, 7]
[[2, 12], [5, 19]]
2.0
1.0
0.0
1.0
8.0
2.0
1.0
0.0
1.0
8.0
1102
[7]
[[2, 5]]
1103
[6]
[[5]]
1104
[6]
[[5]]
1105
[6, 7]
[[5], [2, 5]]
1106
[3, 6]
[[2, 11], [5]]
3.0
0.0
0.0
1.0
9.0
1107
[6]
[[5, 17]]
2.0
1.0
0.0
1.0
8.0
1108
[3, 6]
[[2, 11], [5, 15]]
3.0
0.0
0.0
1.0
9.0
2.0
1.0
0.0
1.0
8.0
1109
[]
1110
[7]
[[2, 4]]
1111
[6]
[[4, 17]]
3.0
0.0
0.0
1.0
9.0
1112
[6]
[[16, 4]]
3.0
0.0
0.0
1.0
9.0
1113
[]
1114
[7]
[[5, 19]]
3.0
0.0
0.0
1.0
9.0
1115
[4]
[[2, 5]]
1116
[3, 7]
[[2, 11], [5, 17]]
3.0
0.0
0.0
1.0
9.0
3.0
0.0
0.0
1.0
9.0
1117
[]
1118
[7]
[[2, 4]]
1119
[5]
[[4, 13]]
3.0
0.0
0.0
1.0
9.0
1120
[1, 5]
[[2, 8], [4, 11]]
3.0
0.0
0.0
1.0
9.0
3.0
0.0
0.0
1.0
9.0
1121
[]
1122
[]
1123
[]
1124
[]
1125
[]
1126
[3]
[[2, 11]]
3.0
0.0
0.0
1.0
9.0
1127
[3]

[5, 7]
[[3, 6], [6]]
1391
[7]
[[6]]
1392
[7]
[[6]]
1393
[5, 7]
[[4], [6, 13]]
1.0
2.0
0.0
1.0
7.0
1394
[4]
[[3]]
1395
[4]
[[3]]
1396
[2, 4]
[[1, 3], [3]]
1397
[4]
[[3]]
1398
[4]
[[3]]
1399
[4]
[[3]]
1400
[4]
[[3]]
1401
[2, 4]
[[1, 3], [3]]
1402
[2, 4]
[[1, 3], [3]]
1403
[1, 4]
[[2, 8], [3]]
1.0
2.0
0.0
1.0
7.0
1404
[1, 4]
[[2, 8], [3]]
1.0
2.0
0.0
1.0
7.0
1405
[7]
[[6]]
1406
[4, 7]
[[3], [6]]
1407
[4]
[[3]]
1408
[4, 7]
[[3], [6]]
1409
[4, 5]
[[3], [3, 6]]
1410
[4, 7]
[[3], [6]]
1411
[4]
[[3]]
1412
[4, 7]
[[3], [6]]
1413
[4, 5]
[[3], [6, 3]]
1414
[2, 7]
[[1, 3], [6]]
1415
[2, 4, 7]
[[1, 3], [3], [6]]
1416
[2, 4]
[[1, 3], [3]]
1417
[2, 4, 7]
[[1, 3], [3], [6]]
1418
[4, 7]
[[3], [6]]
1419
[4, 5]
[[3], [3, 6]]
1420
[1, 7]
[[8, 2], [6]]
1.0
2.0
0.0
1.0
7.0
1421
[1, 4, 7]
[[8, 2], [3], [6]]
1.0
2.0
0.0
1.0
7.0
1422
[1, 4]
[[2, 8], [3]]
1.0
2.0
0.0
1.0
7.0
1423
[1, 4, 7]
[[2, 8], [3], [6]]
1.0
2.0
0.0
1.0
7.0
1424
[1, 4, 5]
[[2, 8], [3], [3, 6]]
1.0
2.0
0.0
1.0
7.0
1425
[7]
[[6, 17]]
1.0
2.0


...

[]
1949
[7]
[[5, 3]]
1950
[]
1951
[]
1952
[]
1953
[7]
[[5, 3]]
1954
[]
1955
[1]
[[2]]
1956
[1, 7]
[[2], [3, 5]]
1957
[1]
[[2]]
1958
[]
1959
[7]
[[5, 3]]
1960
[]
1961
[2]
[[1, 3]]
1962
[2]
[[1, 3]]
1963
[1]
[[2, 8]]
1.0
2.0
0.0
1.0
7.0
1964
[1, 7]
[[8, 2], [3, 5]]
1.0
2.0
0.0
1.0
7.0
1965
[1]
[[2, 8]]
1.0
2.0
0.0
1.0
7.0
1966
[7]
[[4, 2]]
1967
[1]
[[2]]
1968
[1]
[[2]]
1969
[1, 7]
[[2], [2, 4]]
1970
[1]
[[2]]
1971
[1, 6]
[[2], [4, 5]]
1972
[]
1973
[7]
[[2, 4]]
1974
[6]
[[4, 5]]
1975
[]
1976
[6]
[[4, 5]]
1977
[1]
[[8, 2]]
2.0
1.0
0.0
1.0
8.0
1978
[1]
[[8, 2]]
2.0
1.0
0.0
1.0
8.0
1979
[1, 6]
[[8, 2], [4, 5]]
2.0
1.0
0.0
1.0
8.0
1980
[]
1981
[]
1982
[6]
[[5, 7]]
1983
[]
1984
[4]
[[2, 5]]
1985
[3]
[[2, 11]]
3.0
0.0
0.0
1.0
9.0
1986
[3]
[[2, 11]]
3.0
0.0
0.0
1.0
9.0
1987
[3, 6]
[[2, 11], [5, 7]]
3.0
0.0
0.0
1.0
9.0
1988
[3]
[[2, 11]]
3.0
0.0
0.0
1.0
9.0
1989
[]
1990
[7]
[[4, 6]]
1991
[3]
[[2, 11]]
3.0
0.0
0.0
1.0
9.0
1992
[3, 7]
[[2, 11], [4, 6]]
3.0
0.0
0.0
1.0
9.0
1993
[]
1994
[6]
[[5, 7]]


[]
2258
[]
2259
[3]
[[2, 4]]
2260
[3]
[[2, 4]]
2261
[]
2262
[]
2263
[1]
[[8, 2]]
2.0
1.0
0.0
1.0
8.0
2264
[1]
[[8, 2]]
2.0
1.0
0.0
1.0
8.0
2265
[7]
[[6]]
2266
[7]
[[6]]
2267
[5, 7]
[[4, 6], [6]]
2268
[7]
[[6]]
2269
[7]
[[6]]
2270
[3, 7]
[[2, 4], [6]]
2271
[7]
[[6]]
2272
[1, 7]
[[8, 2], [6]]
2.0
1.0
0.0
1.0
8.0
2273
[1, 7]
[[8, 2], [6]]
2.0
1.0
0.0
1.0
8.0
2274
[1, 5]
[[2, 8], [4, 6]]
2.0
1.0
0.0
1.0
8.0
2275
[1, 5, 7]
[[2, 8], [4, 6], [6]]
2.0
1.0
0.0
1.0
8.0
2276
[7]
[[6, 19]]
1.0
2.0
0.0
1.0
7.0
2277
[7]
[[6, 18]]
1.0
2.0
0.0
1.0
7.0
2278
[7]
[[6, 18]]
1.0
2.0
0.0
1.0
7.0
2279
[3, 7]
[[2, 4], [6, 17]]
1.0
2.0
0.0
1.0
7.0
2280
[]
2281
[7]
[[6, 19]]
2.0
1.0
0.0
1.0
8.0
2282
[]
2283
[5]
[[4, 6]]
2284
[]
2285
[3]
[[2, 4]]
2286
[3, 7]
[[2, 4], [17, 6]]
2.0
1.0
0.0
1.0
8.0
2287
[7]
[[6, 18]]
1.0
2.0
0.0
1.0
7.0
2288
[2]
[[1, 3]]
2289
[2, 7]
[[1, 3], [6, 17]]
2.0
1.0
0.0
1.0
8.0
2290
[2]
[[1, 3]]
2291
[2]
[[1, 3]]
2292
[2, 5]
[[1, 3], [4, 6]]
2293
[1, 7]
[[2, 8], [6, 17]]
1.0
2.0
0.0
1.0
7.

[5, 7]
[[4, 6], [6]]
2595
[4]
[[3, 5]]
2596
[4]
[[3, 5]]
2597
[4, 7]
[[3, 5], [6]]
2598
[7]
[[6, 13]]
1.0
2.0
0.0
1.0
7.0
2599
[]
2600
[]
2601
[]
2602
[1]
[[2, 8]]
1.0
2.0
0.0
1.0
7.0
2603
[1]
[[2, 8]]
1.0
2.0
0.0
1.0
7.0
2604
[7]
[[6]]
2605
[1, 7]
[[8, 2], [6]]
1.0
2.0
0.0
1.0
7.0
2606
[2]
[[1, 3]]
2607
[2]
[[1, 3]]
2608
[7]
[[6]]
2609
[5, 7]
[[6, 4], [6]]
2610
[2, 7]
[[3, 1], [6]]
2611
[]
2612
[]
2613
[2]
[[1, 3]]
2614
[2]
[[1, 3]]
2615
[1]
[[2, 8]]
1.0
2.0
0.0
1.0
7.0
2616
[1]
[[2, 8]]
1.0
2.0
0.0
1.0
7.0
2617
[7]
[[6]]
2618
[2, 7]
[[1, 3], [6]]
2619
[1, 7]
[[2, 8], [6]]
1.0
2.0
0.0
1.0
7.0
2620
[7]
[[6, 17]]
1.0
2.0
0.0
1.0
7.0
2621
[6]
[[5, 7]]
2622
[]
2623
[7]
[[17, 6]]
1.0
2.0
0.0
1.0
7.0
2624
[2, 7]
[[1, 3], [15, 6]]
1.0
2.0
0.0
1.0
7.0
2625
[1, 7]
[[2, 8], [15, 6]]
1.0
2.0
0.0
1.0
7.0
1.0
2.0
0.0
1.0
7.0
2626
[6]
[[5, 7]]
2627
[3]
[[2, 4]]
2628
[2, 7]
[[3, 1], [16, 6]]
1.0
2.0
0.0
1.0
7.0
2629
[7]
[[6, 17]]
1.0
2.0
0.0
1.0
7.0
2630
[]
2631
[3]
[[2, 4]]
2632
[2]
[[1, 3]]
2633
[

[]
2919
[]
2920
[]
2921
[7]
[[5, 6]]
2922
[1]
[[2, 8]]
2.0
1.0
0.0
1.0
8.0
2923
[1]
[[2, 8]]
2.0
1.0
0.0
1.0
8.0
2924
[1, 7]
[[2, 8], [5, 6]]
2.0
1.0
0.0
1.0
8.0
2925
[1]
[[8, 2]]
2.0
1.0
0.0
1.0
8.0
2926
[1]
[[8, 2]]
2.0
1.0
0.0
1.0
8.0
2927
[1, 7]
[[2, 8], [5, 6]]
2.0
1.0
0.0
1.0
8.0
2928
[4]
[[3]]
2929
[4]
[[3]]
2930
[4, 7]
[[3], [5, 6]]
2931
[4]
[[3]]
2932
[4]
[[3]]
2933
[1, 4]
[[2, 8], [3]]
2.0
1.0
0.0
1.0
8.0
2934
[1, 4]
[[2, 8], [3]]
2.0
1.0
0.0
1.0
8.0
2935
[1, 4, 7]
[[2, 8], [3], [5, 6]]
2.0
1.0
0.0
1.0
8.0
2936
[4]
[[3]]
2937
[4]
[[3]]
2938
[4, 7]
[[3], [6, 5]]
2939
[4]
[[3]]
2940
[4]
[[3]]
2941
[4]
[[3]]
2942
[4, 7]
[[3], [5, 6]]
2943
[4]
[[3]]
2944
[2, 4]
[[1, 3], [3]]
2945
[2, 4]
[[1, 3], [3]]
2946
[2, 4, 7]
[[1, 3], [3], [5, 6]]
2947
[2, 4]
[[3, 1], [3]]
2948
[1, 4]
[[2, 8], [3]]
1.0
2.0
0.0
1.0
7.0
2949
[1, 4]
[[8, 2], [3]]
1.0
2.0
0.0
1.0
7.0
2950
[1, 4, 7]
[[2, 8], [3], [6, 5]]
1.0
2.0
0.0
1.0
7.0
2951
[1, 4]
[[8, 2], [3]]
1.0
2.0
0.0
1.0
7.0
2952
[4]
[[3, 14]]
1.0
2.0

0